In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random

from multiprocessing.dummy import Pool as ThreadPool
import itertools
from collections import defaultdict

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *

Using gpu device 0: Tesla K40m (CNMeM is disabled, cuDNN 5105)
/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
IS_SAMPLE = False

In [4]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234
WORD2VEC_SEED = 1234

In [5]:
NUMBER_INDICATOR = "number_inidicator"
CURRENCY_INDICATOR = "currency_inidicator"
CHEMICAL_INDICATOR = "chemical_inidicator"
MIN_WORD_COUNT = 100
MIN_SIZE = 0
NUM_CORES = 14

In [6]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [7]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_MATRIX = "validation_matrix.pkl"
TEST_MATRIX = "test_matrix.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"

In [8]:
NN_PARAMETER_SEARCH_PREFIX = "{}_batch_{}_nn_parameter_searches.pkl"

In [9]:
#training_file = "/home/local/shalaby/docs_output_sample_100.json"

root_location = "/mnt/data2/shalaby/"
exports_location = root_location + "exported_data/"

doc2vec_model_save_location = os.path.join(root_location, "parameter_search_doc2vec_models_new", "full")
nn_parameter_search_location = os.path.join(root_location, "nn_parameter_search")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

training_file = root_location + "docs_output.json"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
classification_index_file = exports_location + "classification_index.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
valid_classes_file = exports_location + "valid_classes.pkl"
valid_subclasses_file = exports_location + "valid_subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/"

training_preprocessed_files_prefix = preprocessed_location + "training_docs_merged_data_preprocessed-"
training_preprocessed_docids_files_prefix = preprocessed_location + "training_docs_merged_docids_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "validation_docs_merged_data_preprocessed-"
validation_preprocessed_docids_files_prefix = preprocessed_location + "validation_docs_merged_docids_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "test_docs_merged_data_preprocessed-"
test_preprocessed_docids_files_prefix = preprocessed_location + "test_docs_merged_docids_preprocessed-"

word2vec_questions_file = result = root_location + 'tensorflow/word2vec/questions-words.txt'

In [10]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
valid_classes = pickle.load(open(valid_classes_file))
valid_subclasses = pickle.load(open(valid_subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
# classifications_index = pickle.load(open(classification_index_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 16.7 s, sys: 764 ms, total: 17.4 s
Wall time: 17.4 s


In [11]:
len(training_docs_list)

1286325

In [12]:
len(validation_docs_list)

321473

In [13]:
len(test_docs_list)

401877

In [14]:
def ensure_disk_location_exists(location):
    if not os.path.exists(location):
        os.makedirs(location)

In [37]:
def get_docs_with_inference(doc2vec_model, doc_classification_map, classifications,
                                           docs_list, file_to_write, preprocessed_files_prefix,
                                           preprocessed_docids_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)


    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_labels = []
        inference_vectors_matrix = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
        for doc_id in docs_list:
            curr_doc_labels = set(doc_classification_map[doc_id]) & classifications_set
            inference_labels.append(one_hot_encoder.get_label_vector(curr_doc_labels))
        inference_labels = np.array(inference_labels, dtype=np.int8)
    else:
        inference_documents_reps = {}
        inference_vectors = []
        inference_labels = []
        info("===== Getting vectors with inference")


        # Multi-threaded inference
        inference_docs_iterator = DocumentBatchGenerator(preprocessed_files_prefix,
                                                          preprocessed_docids_files_prefix, batch_size=None)
        generator_func = inference_docs_iterator.__iter__()
        pool = ThreadPool(NUM_CORES)
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = 1000
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {}".format(str(inference_docs_iterator.curr_index)))
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break

        # create matrix for the inferred vectors
        for doc_id in docs_list:
            inference_vectors.append(inference_documents_reps[doc_id])
            curr_doc_labels = set(doc_classification_map[doc_id]) & classifications_set
            inference_labels.append(one_hot_encoder.get_label_vector(curr_doc_labels))
        inference_vectors_matrix = np.array(inference_vectors)
        inference_labels = np.array(inference_labels, dtype=np.int8)
        pickle.dump(inference_vectors_matrix,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_vectors_matrix, inference_labels

In [16]:
def get_validation_docs_with_inference(doc2vec_model, doc_classification_map):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation documents
    """
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX)):
        info("===== Loading validation vectors")
        validation_vectors_matrix = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX)))
    else:
        validation_documents_reps = {}
        validation_vectors = []
        validation_labels = []
        info("===== Getting validation vectors with inference")

        # do inference and store results in dict
        i = 0
        for (doc_id, doc_contents_array) in ValidationDocumentGenerator(training_file, validation_docs_list):
            i += 1
            if i % 1000 == 0: info("Finished: {}".format(str(i)))
            validation_documents_reps[doc_id] = doc2vec_model.infer_vector(doc_contents_array)

        # create matrix for the validation vectors
        for validation_doc_id in validation_docs_list:
            validation_vectors.append(validation_documents_reps[validation_doc_id])
            validation_labels.append([classf for classf in doc_classification_map[validation_doc_id] if classf in sections])
        validation_vectors_matrix = np.array(validation_vectors)
        pickle.dump(validation_vectors_matrix, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX), 'w'))
    
    return validation_vectors_matrix

In [17]:
def get_validation_docs_with_inference_new(doc2vec_model, doc_classification_map, classifications, 
                                           val_docs_list, val_preprocessed_files_prefix, val_preprocessed_docids_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation documents
    """

    def infer_one_doc(doc_tuple):
        #doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX)):
        info("===== Loading validation vectors")
        validation_labels = []
        validation_vectors_matrix = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX)))
        info("Loaded validation vectors matrix")
        for i, validation_doc_id in enumerate(val_docs_list):
#             val_labels = [classf for classf in doc_classification_map[validation_doc_id] if classf in classifications]
            val_labels = list(set(doc_classification_map[validation_doc_id]) & classifications_set)
            validation_labels.append(one_hot_encoder.get_label_vector(val_labels))
            if i % 100000 == 0:
                info("Finished {} in validation loading".format(i))
        validation_labels = np.array(validation_labels, dtype=np.int8)
    else:
        validation_documents_reps = {}
        validation_vectors = []
        validation_labels = []
        info("===== Getting validation vectors with inference")

        # Single-threaded inference
        # do inference and store results in dict
#         i = 0
        
#         validation_docs_iterator = DocumentBatchGenerator(val_preprocessed_files_prefix, 
#                                                         val_preprocessed_docids_files_prefix, batch_size=None)
#         for (doc_id, doc_contents_array) in validation_docs_iterator:
#             i += 1
#             if i % 1000 == 0: info("Finished: {}".format(str(i)))
#             validation_documents_reps[doc_id] = doc2vec_model.infer_vector(doc_contents_array)
        
        # Multi-threaded inference
        validation_docs_iterator = DocumentBatchGenerator(validation_preprocessed_files_prefix, 
                                                          validation_preprocessed_docids_files_prefix, batch_size=None)
        generator_func = validation_docs_iterator.__iter__()
        pool = ThreadPool(NUM_CORES)
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        validation_documents_reps = {}
        mini_batch_size = 1000
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {}".format(str(validation_docs_iterator.curr_index)))
            if threaded_reps_partial:
                #threaded_reps.extend(threaded_reps_partial)
                validation_documents_reps.update(threaded_reps_partial)
            else:
                break

                
        # create matrix for the validation vectors
        for validation_doc_id in val_docs_list:
            validation_vectors.append(validation_documents_reps[validation_doc_id])
#             val_labels = [classf for classf in doc_classification_map[validation_doc_id] if classf in classifications]
            val_labels = set(doc_classification_map[validation_doc_id]) & classifications_set
            validation_labels.append(one_hot_encoder.get_label_vector(val_labels))
        validation_vectors_matrix = np.array(validation_vectors)
        validation_labels = np.array(validation_labels, dtype=np.int8)
        pickle.dump(validation_vectors_matrix, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_MATRIX), 'w'))
    
    return validation_vectors_matrix, validation_labels

In [18]:
class OneHotEncoder():
    
    def __init__(self, classifications):
        self.classifications = classifications
        self.one_hot_indices = {}

        # convert character classifications to bit vectors
        for i, clssf in enumerate(classifications):
            bits = [0] * len(classifications)
            bits[i] = 1
            self.one_hot_indices[clssf] = i
    
    def get_label_vector(self, labels):
        """
        classes: array of string with the classes assigned to the instance
        """
        output_vector = [0] * len(self.classifications)
        for label in labels:
            index = self.one_hot_indices[label]
            output_vector[index] = 1
            
        return output_vector

def get_training_data(doc2vec_model, classifications):
    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    training_data = []
    training_labels_mat = np.zeros((len(training_docs_list), len(classifications)), dtype=np.int8)
#     training_data_mat = np.zeros((len(training_docs_list), DOC2VEC_SIZE), dtype=np.float32)
    for i,doc_id in enumerate(training_docs_list):
        # converting from memmap to a normal array
#         normal_array = []
#         normal_array[:] = doc2vec_model.docvecs[doc_id][:]
        normal_array = doc2vec_model.docvecs[doc_id]
        training_data.append(normal_array)
#         eligible_classifications = [clssf for clssf in doc_classification_map[doc_id] if clssf in classifications]
        eligible_classifications = set(doc_classification_map[doc_id]) & classifications_set
        training_labels_mat[i][:] = one_hot_encoder.get_label_vector(eligible_classifications)
        if i % 100000 == 0:
            info("Finished {} in training".format(i))
    info("doing matrix creation")
    training_data_mat = np.array(training_data)
#     training_labels_mat = np.array(training_labels, dtype=np.int8)
    del training_data
    return training_data_mat, training_labels_mat

In [19]:
class TrainingDocumentGenerator(object):
    def __init__(self, filename, training_docs_list):
        self.filename = filename
        self.training_docs_list = training_docs_list
    def __iter__(self):
        with open(self.filename) as file_obj:
            for line in file_obj:
                if not line.strip(): continue
                (doc_id, text) = eval(line)
                if doc_id in self.training_docs_list:
                    yield LabeledSentence(words=stemtokenizer(text), tags=[doc_id])
                    
class DocumentBatchGenerator(object):
    def __init__(self, filename_prefix, filename_docids_prefix, batch_size=10000 ):
        """
        batch_size cant be > 10,000 due to a limitation in doc2vec training, 
        None means no batching (only use for inference)
        """
        assert batch_size <= 10000 or batch_size is None
        self.filename_prefix = filename_prefix
        self.filename_docids_prefix = filename_docids_prefix
        self.curr_lines = []
        self.curr_docids = []
        self.batch_size = batch_size
        self.curr_index = 0
        self.batch_end = -1
    def load_new_batch_in_memory(self):
        del self.curr_lines, self.curr_docids
        self.curr_lines, self.docids = [], []
        info("Loading new batch for index: {}".format(self.curr_index) )
        try:
            with open(self.filename_prefix + str(self.curr_index)) as preproc_file:
                for line in preproc_file:
                    self.curr_lines.append(line.split(" "))
#                     if i % 1000 == 0:
#                         print i
            self.curr_docids = pickle.load(open(self.filename_docids_prefix + str(self.curr_index), "r"))
            self.batch_end = self.curr_index + len(self.curr_lines) -1 
            info("Finished loading new batch")
        except IOError:
            info("No more batches to load, exiting at index: {}".format(self.curr_index))
            raise StopIteration()
    def __iter__(self):
        while True:
            if self.curr_index > self.batch_end:
                self.load_new_batch_in_memory()
            for (doc_id, tokens) in zip(self.curr_docids, self.curr_lines):
                if self.batch_size is not None:
                    curr_batch_iter = 0
                    # divide the document to batches according to the batch size
                    while curr_batch_iter < len(tokens):
                        yield LabeledSentence(words=tokens[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id])
                        curr_batch_iter += self.batch_size
                else:
                    yield doc_id, tokens
                self.curr_index += 1

class Word2VecTrainingDocumentGenerator(object):
    def __init__(self, filename, training_docs_list):
        self.filename = filename
        self.training_docs_list = training_docs_list
    def __iter__(self):
        with open(self.filename) as file_obj:
            for line in file_obj:
                if not line.strip(): continue
                (doc_id, text) = eval(line)
                if doc_id in self.training_docs_list:
                    yield stemtokenizer(text)
                
class ValidationDocumentGenerator(object):
    def __init__(self, filename, validation_docs_list):
        self.filename = filename
        self.validation_docs_list = validation_docs_list
    def __iter__(self):
        with open(self.filename) as file_obj:
            for line in file_obj:
                if not line.strip(): continue
                (doc_id, text) = eval(line)
                if doc_id in self.validation_docs_list:
                    yield doc_id, stemtokenizer(text)
                    
class StochasticDocumentGenerator(object):
    """
    Randomly shuffle rows while reading them
    """
    def __init__(self, filename, training_docs_list, line_positions):
        self.filename = filename
        self.training_docs_list = training_docs_list
        self.line_positions = line_positions
        self.lines = set(line_positions.keys())
    def __iter__(self):
        with open(self.filename) as file_obj:
            while len(self.lines) > 0:
                random_line = random.sample(self.lines,1)[0]
                self.lines.remove(random_line)
                file_obj.seek(self.line_positions[random_line])
                line = file_obj.readline()
                if not line.strip(): continue
#                 print random_line, self.line_positions[random_line], line[:30]
                (doc_id, text) = eval(line)
                # print random_line , doc_id
                if doc_id in self.training_docs_list:
                    yield LabeledSentence(words=stemtokenizer(text), tags=[doc_id])
#                     yield doc_id

## Loading Specific Doc2vec Model

### Doc2vec and SVM Parameters

In [21]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 20
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 10000 # report vocab progress every x documents

In [22]:
placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}'.format(DOC2VEC_SIZE, 
                                                                DOC2VEC_WINDOW, 
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE))
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
placeholder_model_name

'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_{}'

In [23]:
doc2vec_methods_dict = {
#     'doc2vec_size_50_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None' : 7,
#     'doc2vec_size_50_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None' : 8,
#     'doc2vec_size_50_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_100_w_2_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 9,
#     'doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 18,
#     'doc2vec_size_100_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 10,
#     'doc2vec_size_100_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 4,
#     'doc2vec_size_100_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 7,
#     'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_200_w_2_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 7,
#     'doc2vec_size_200_w_4_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 6,
#     'doc2vec_size_200_w_4_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_200_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 14,
#     'doc2vec_size_200_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 1,
#     'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_500_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_500_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_1000_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 6,
#     'doc2vec_size_1000_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 5
}

In [25]:
classifications = sections
classifications_type = 'sections'

In [26]:
epoch = 8
GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
info("****************** Epoch {} --- Loading {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))

# if we have the model, just load it, otherwise train the previous model
if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
    doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
    GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

info('Getting training Data')
X, y = get_training_data(doc2vec_model, classifications)

info('Getting Validation Embeddings')
Xv, yv = get_validation_docs_with_inference_new(doc2vec_model, doc_classification_map, classifications, 
                                                validation_docs_list, validation_preprocessed_files_prefix,
                                                validation_preprocessed_docids_files_prefix)

2017-02-27 11:29:21,274 : INFO : ****************** Epoch 8 --- Loading doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8 *******************
2017-02-27 11:29:21,276 : INFO : loading Doc2Vec object from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model
2017-02-27 11:29:27,608 : INFO : loading docvecs recursively from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.* with mmap=None
2017-02-27 11:29:27,609 : INFO : loading doctag_syn0 from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.doctag_syn0.npy with mmap=None
2017-02-27 11:29:29,781 : INFO : loading syn1neg from /mnt/data2/shalaby/parameter_search

In [27]:
# create nn parameter search directory
if not os.path.exists(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME)):
    os.makedirs(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME))

## NN Specific Functions

In [28]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from sklearn.model_selection import ParameterSampler
import keras

Using Theano backend.


In [29]:
def create_keras_nn_model(input_size, output_size, 
                          first_hidden_layer_size, first_hidden_layer_activation, 
                          second_hidden_layer_size, second_hidden_layer_activation, 
                          input_dropout_do, hidden_dropout_do):
    
    doc_input = Input(shape=(DOC2VEC_SIZE,), name='doc_input')
    if input_dropout_do:
        hidden = Dropout(0.7)(doc_input)
    hidden = Dense(first_hidden_layer_size, activation=first_hidden_layer_activation, 
                   name='hidden_layer_{}'.format(first_hidden_layer_activation))(doc_input if not input_dropout_do else hidden)
    if hidden_dropout_do:
        hidden = Dropout(0.5)(hidden)
    if second_hidden_layer_size is not None:
        hidden = Dense(second_hidden_layer_size, activation=second_hidden_layer_activation, 
                       name='hidden_layer2_{}'.format(second_hidden_layer_activation))(hidden)
    softmax_output = Dense(output_size, activation='sigmoid', name='softmax_output')(hidden)

    model = Model(input=doc_input, output=softmax_output)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')
    
    return model

In [30]:
VALIDATION_METRICS_FILENAME= '{}_validation_metrics.pkl'.format(classifications_type)
TRAINING_METRICS_FILENAME = '{}_training_metrics.pkl'.format(classifications_type)
TEST_METRICS_FILENAME = '{}_test_metrics.pkl'.format(classifications_type)
METRICS_FIG_PNG_FILENAME = '{}_validation_metrics.png'.format(classifications_type)
METRICS_FIG_PDF_FILENAME = '{}_validation_metrics.pdf'.format(classifications_type)
WORD2VEC_METRICS_FILENAME = 'word2vec_metrics.pkl'

# for epoch in range(DOC2VEC_MAX_EPOCHS):
#     GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
#     ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
#                                              GLOBAL_VARS.SVM_MODEL_NAME))
#     pickle.dump(metrics, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, GLOBAL_VARS.SVM_MODEL_NAME, METRICS), 'w'))
# fig_save_location = placeholder_model_name.format('run')
# plt.savefig(os.path.join(fig_save_location))

In [31]:
get_binary_0_5 = lambda x: 1 if x > 0.5 else 0
get_binary_0_5 = np.vectorize(get_binary_0_5)

In [30]:
NN_OUTPUT_NEURONS = len(classifications)
NN_MAX_EPOCHS = 100
NN_BATCH_SIZE = 1024
NN_RANDOM_SEARCH_BUDGET = 30
NN_PARAM_SAMPLE_SEED = 1234

first_hidden_layer_sizes = [200,500]
second_hidden_layer_sizes = [None,50,200,500,1000,2000]
first_hidden_layer_activations = ['relu','sigmoid', 'tanh', 'softmax']
second_hidden_layer_activations = ['relu','sigmoid', 'tanh', 'softmax']
# first_hidden_layer_activations = ['relu']
# second_hidden_layer_activations = ['relu']
input_dropout_options = [False, True]
hidden_dropout_options = [False, True]

## Evaluate Batch Sizes

In [65]:
batch_dict = {}
for batch_sz in [64,128,256,512,1024,2048]:
# for batch_sz in [2048]:
    NN_BATCH_SIZE = batch_sz
    batch_dict[batch_sz] = {}
    start_time = time.time()
    param_sampler = ParameterSampler({
            'first_hidden_layer_size':first_hidden_layer_sizes,
            'first_hidden_layer_activation':first_hidden_layer_activations,
            'second_hidden_layer_size':second_hidden_layer_sizes,
            'second_hidden_layer_activation':second_hidden_layer_activations,
            'input_dropout':input_dropout_options,
            'hidden_dropout':hidden_dropout_options
        }, n_iter=NN_RANDOM_SEARCH_BUDGET, random_state=NN_PARAM_SAMPLE_SEED)
    for parameters in param_sampler:
        first_hidden_layer_size = parameters['first_hidden_layer_size']
        first_hidden_layer_activation = parameters['first_hidden_layer_activation']
        second_hidden_layer_size = parameters['second_hidden_layer_size']
        second_hidden_layer_activation = parameters['second_hidden_layer_activation']
        input_dropout_do = parameters['input_dropout']
        hidden_dropout_do = parameters['hidden_dropout']

        print ("===================================================================================\n" + \
              "========== 1st Layer Size: {}, 1st Layer Activation: {}, \n 2nd Layer Size: {}, 2nd Layer Activation: {}, \n" + \
              "Input Dropout: {}, Hidden Dropout: {} \n" + \
              "==========================").format(first_hidden_layer_size, first_hidden_layer_activation, 
                                                    second_hidden_layer_size, second_hidden_layer_activation, 
                                                    input_dropout_do, hidden_dropout_do)

        GLOBAL_VARS.NN_MODEL_NAME = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
            first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
            second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
        )

        model = create_keras_nn_model(DOC2VEC_SIZE, NN_OUTPUT_NEURONS, 
                                      first_hidden_layer_size, first_hidden_layer_activation, 
                                      second_hidden_layer_size, second_hidden_layer_activation, 
                                      input_dropout_do, hidden_dropout_do)
        model.summary()
        
        batch_dict[batch_sz]['metrics'] = []
        batch_dict[batch_sz]['history'] = []

        for nn_epoch in range(1, NN_MAX_EPOCHS+1):
            info('======= NN Epoch: {} =========='.format(nn_epoch)) 
            %time history = model.fit(x=X, y=y, validation_data=(Xv,yv), batch_size=NN_BATCH_SIZE, nb_epoch=1, verbose=1, callbacks=[early_stopper])
    #         info('Evaluating on Training Data')
    #         yp = model.predict(X, batch_size=NN_BATCH_SIZE)
    #         yp_binary = get_binary_0_5(yp)
    #         #print yp
    #         info('Generating Training Metrics')
    #         training_metrics = get_metrics(y, yp, yp_binary)
    #         print "** Training Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
    #             training_metrics['coverage_error'], training_metrics['average_num_of_labels'], 
    #             training_metrics['top_1'], training_metrics['top_3'], training_metrics['top_5'], 
    #             training_metrics['f1_micro'],training_metrics['f1_macro'],  training_metrics['total_positive'])

            info('Evaluating on Validation Data')
            yvp = model.predict(Xv)
            yvp_binary = get_binary_0_5(yvp)
            #print yvp
            info('Generating Validation Metrics')
            validation_metrics = get_metrics(yv, yvp, yvp_binary)
            print "****** Validation Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
                validation_metrics['coverage_error'], validation_metrics['average_num_of_labels'], 
                validation_metrics['top_1'], validation_metrics['top_3'], validation_metrics['top_5'], 
                validation_metrics['f1_micro'], validation_metrics['f1_macro'], validation_metrics['total_positive'])
            
            batch_dict[batch_sz]['metrics'].append(validation_metrics)
            batch_dict[batch_sz]['history'].append(history)
            
        break
    duration = time.time() - start_time
    batch_dict[batch_sz]['duration'] =  duration


========== 1st Layer Size: 200, 1st Layer Activation: tanh, 
 2nd Layer Size: 500, 2nd Layer Activation: tanh, 
Input Dropout: True, Hidden Dropout: True 
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
doc_input (InputLayer)           (None, 100)           0                                            
____________________________________________________________________________________________________
dropout_29 (Dropout)             (None, 100)           0           doc_input[0][0]                  
____________________________________________________________________________________________________
hidden_layer_tanh (Dense)        (None, 200)           20200       dropout_29[0][0]                 
____________________________________________________________________________________________________
dropout_30 (Dropout)             (Non

2017-01-22 23:18:08,422 : INFO : ======= NN Epoch: 1 ==========


Total params: 124708
____________________________________________________________________________________________________
Train on 1286325 samples, validate on 321473 samples
Epoch 1/1
1286325/1286325 [==============================] - 749s - loss: 0.2799 - val_loss: 0.2113

2017-01-22 23:30:46,709 : INFO : Evaluating on Validation Data



CPU times: user 1min 22s, sys: 52.7 s, total: 2min 15s
Wall time: 12min 37s


...

****** Validation Metrics: Cov Err: 1.625, Avg Labels: 1.150, 
		 Top 1: 0.738, Top 3: 0.948, Top 5: 0.988, 
		 F1 Micro: 0.651, F1 Macro: 0.464, Total Pos: 267,543
Train on 1286325 samples, validate on 321473 samples
Epoch 1/1
1286325/1286325 [==============================] - 20s - loss: 0.2630 - val_loss: 0.1998

2017-01-23 02:32:11,581 : INFO : Evaluating on Validation Data



CPU times: user 11.4 s, sys: 9.79 s, total: 21.2 s
Wall time: 20.3 s


2017-01-23 02:32:18,510 : INFO : Generating Validation Metrics


****** Validation Metrics: Cov Err: 1.676, Avg Labels: 1.150, 
		 Top 1: 0.731, Top 3: 0.935, Top 5: 0.983, 
		 F1 Micro: 0.677, F1 Macro: 0.433, Total Pos: 308,801


In [92]:
for sz in sorted(batch_dict.keys()):
    print '========== Batch Size: {}'.format(sz)
    print 'Duration => {}'.format(batch_dict[sz]['duration'])
    print 'Loss History => {}'.format([round(hist.history['loss'][0],3) for hist in batch_dict[sz]['history']])
    print 'Val Loss Hist => {}'.format([round(hist.history['val_loss'][0],3) for hist in batch_dict[sz]['history']])
    cov_error_history = [round(metrics['coverage_error'],3) for metrics in batch_dict[sz]['metrics']]
    f1_micro_history = [round(metrics['f1_micro'],3) for metrics in batch_dict[sz]['metrics']]
    f1_macro_history = [round(metrics['f1_macro'],3) for metrics in batch_dict[sz]['metrics']]
    print 'Cov Erro Hist=> {}'.format(cov_error_history)
    print 'F1 Micro Hist=> {}'.format(f1_micro_history)
    print 'F1 Macro Hist=> {}'.format(f1_macro_history)
    print 'Coverage Error Max: {} => {} | F1 Micro Max: {} => {} | F1 Macro Max: {} => {}'.format(np.argmin(cov_error_history),
                                                                                                np.min(cov_error_history),
                                                                                                np.argmax(f1_micro_history),
                                                                                                np.max(f1_micro_history), 
                                                                                                np.argmax(f1_macro_history),
                                                                                                np.max(f1_macro_history))

========== Batch Size: 64
Duration => 2902.9390409
Loss History => [0.28, 0.273, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272, 0.272]
Val Loss Hist => [0.211, 0.207, 0.208, 0.208, 0.203, 0.203, 0.206, 0.206, 0.204, 0.204, 0.205, 0.207, 0.21, 0.205, 0.206]
Cov Erro Hist=> [1.709, 1.664, 1.684, 1.688, 1.675, 1.676, 1.708, 1.676, 1.66, 1.665, 1.669, 1.691, 1.711, 1.676, 1.7]
F1 Micro Hist=> [0.633, 0.627, 0.634, 0.631, 0.647, 0.648, 0.641, 0.65, 0.642, 0.658, 0.65, 0.632, 0.605, 0.637, 0.663]
F1 Macro Hist=> [0.416, 0.455, 0.436, 0.433, 0.446, 0.434, 0.439, 0.46, 0.459, 0.435, 0.482, 0.423, 0.421, 0.451, 0.458]
Coverage Error Max: 8 => 1.66 | F1 Micro Max: 14 => 0.663 | F1 Macro Max: 10 => 0.482
========== Batch Size: 128
Duration => 2890.55074692
Loss History => [0.28, 0.271, 0.27, 0.269, 0.269, 0.269, 0.268, 0.268, 0.268, 0.268, 0.268, 0.268, 0.268, 0.268, 0.268]
Val Loss Hist => [0.209, 0.203, 0.203, 0.202, 0.202, 0.203, 0.203, 0.208, 0.201, 0.201,

In [186]:
pickle.dump(batch_dict, open('/mnt/data2/shalaby/exported_data/nn_batch_sizes.pkl','w'))

## Actual Parameter searches for NN

In [31]:
early_stopper_deltas = {
    'sections': 0.0001,
    'classes': 0.00001,
    'subclasses': 0.00001
}
early_stopper_patience = {
    'sections': 5,
    'classes': 10,
    'subclasses': 10
}
epochs_before_validation = {
    'sections': 50,
    'classes': 50,
    'subclasses': 50
}

In [32]:
NN_OUTPUT_NEURONS = len(classifications)
EARLY_STOPPER_MIN_DELTA = early_stopper_deltas[classifications_type]
EARLY_STOPPER_PATIENCE = early_stopper_patience[classifications_type]

In [33]:
NN_BATCH_SIZE = 1024

In [34]:
class MetricsCallback(keras.callbacks.Callback):
    
    EPOCHS_BEFORE_VALIDATION = epochs_before_validation[classifications_type]
    
    def on_train_begin(self, logs={}):
        self.epoch_index = 0
        self.val_loss_reductions = 0
        self.metrics_dict = {}
        self.best_val_loss = np.iinfo(np.int32).max
        self.best_weights = None
        self.best_validation_metrics = None
    def on_epoch_end(self, epoch, logs={}):
        self.epoch_index += 1
        if logs['val_loss'] < self.best_val_loss:
            self.val_loss_reductions += 1
            self.best_val_loss = logs['val_loss']
            self.best_weights = self.model.get_weights()
            print '\r    \r' # to remove the previous line of verbose output of model fit
            time.sleep(0.2)
            info('Found lower val loss for epoch {} => {}'.format(self.epoch_index, round(logs['val_loss'], 5)))
            if self.val_loss_reductions % MetricsCallback.EPOCHS_BEFORE_VALIDATION == 0:
                
                info('Validation Loss Reduced {} times'.format(self.val_loss_reductions))
                info('Evaluating on Validation Data')
                yvp = self.model.predict(Xv)
                yvp_binary = get_binary_0_5(yvp)
                info('Generating Validation Metrics')
                validation_metrics = get_metrics(yv, yvp, yvp_binary)
                print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
                    validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
                    validation_metrics['f1_micro'], validation_metrics['f1_macro'])
                self.metrics_dict[self.epoch_index] = validation_metrics
#                 self.best_validation_metrics = validation_metrics


In [66]:
param_sampler = ParameterSampler({
        'first_hidden_layer_size':first_hidden_layer_sizes,
        'first_hidden_layer_activation':first_hidden_layer_activations,
        'second_hidden_layer_size':second_hidden_layer_sizes,
        'second_hidden_layer_activation':second_hidden_layer_activations,
        'input_dropout':input_dropout_options,
        'hidden_dropout':hidden_dropout_options
    }, n_iter=NN_RANDOM_SEARCH_BUDGET, random_state=NN_PARAM_SAMPLE_SEED)

In [51]:
for parameters in param_sampler:
    start_time = time.time()
    first_hidden_layer_size = parameters['first_hidden_layer_size']
    first_hidden_layer_activation = parameters['first_hidden_layer_activation']
    second_hidden_layer_size = parameters['second_hidden_layer_size']
    second_hidden_layer_activation = parameters['second_hidden_layer_activation']
    input_dropout_do = parameters['input_dropout']
    hidden_dropout_do = parameters['hidden_dropout']

    dd = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
        first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
        second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
    )
    print dd
    

nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_tanh_in-drop_True_hid-drop_True
nn_1st-size_500_1st-act_softmax_2nd-size_None_2nd-act_softmax_in-drop_True_hid-drop_False
nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_relu_in-drop_False_hid-drop_True
nn_1st-size_500_1st-act_tanh_2nd-size_50_2nd-act_sigmoid_in-drop_False_hid-drop_True
nn_1st-size_500_1st-act_sigmoid_2nd-size_None_2nd-act_relu_in-drop_True_hid-drop_True
nn_1st-size_200_1st-act_tanh_2nd-size_1000_2nd-act_relu_in-drop_False_hid-drop_True
nn_1st-size_200_1st-act_sigmoid_2nd-size_None_2nd-act_relu_in-drop_False_hid-drop_False
nn_1st-size_500_1st-act_sigmoid_2nd-size_500_2nd-act_sigmoid_in-drop_True_hid-drop_False
nn_1st-size_200_1st-act_sigmoid_2nd-size_None_2nd-act_relu_in-drop_True_hid-drop_True
nn_1st-size_200_1st-act_sigmoid_2nd-size_50_2nd-act_softmax_in-drop_False_hid-drop_False
nn_1st-size_500_1st-act_softmax_2nd-size_None_2nd-act_sigmoid_in-drop_False_hid-drop_False
nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-a

In [63]:
param_results_dict = {}

Load param results if needed to continue

In [35]:
param_results_dict = pickle.load(open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE)))))

In [36]:
to_skip = []
# to_skip = ['nn_1st-size_500_1st-act_softmax_2nd-size_None_2nd-act_softmax_in-drop_True_hid-drop_False', 
#            'nn_1st-size_200_1st-act_sigmoid_2nd-size_50_2nd-act_sigmoid_in-drop_False_hid-drop_True']
to_skip = [
    'nn_1st-size_500_1st-act_tanh_2nd-size_None_2nd-act_sigmoid_in-drop_False_hid-drop_True'
]

In [37]:
for parameters in param_sampler:
    start_time = time.time()
    first_hidden_layer_size = parameters['first_hidden_layer_size']
    first_hidden_layer_activation = parameters['first_hidden_layer_activation']
    second_hidden_layer_size = parameters['second_hidden_layer_size']
    second_hidden_layer_activation = parameters['second_hidden_layer_activation']
    input_dropout_do = parameters['input_dropout']
    hidden_dropout_do = parameters['hidden_dropout']

#     print "===================================================================================\n" + \
#           "========== 1st Layer Size: {}, 1st Layer Activation: {}, \n 2nd Layer Size: {}, 2nd Layer Activation: {}, \n" + \
#           "Input Dropout: {}, Hidden Dropout: {} \n" + \
#           "==========================".format(first_hidden_layer_size, first_hidden_layer_activation, 
#                                                 second_hidden_layer_size, second_hidden_layer_activation, 
#                                                 input_dropout_do, hidden_dropout_do)

    GLOBAL_VARS.NN_MODEL_NAME = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
        first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
        second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
    )
    if GLOBAL_VARS.NN_MODEL_NAME in param_results_dict.keys() or GLOBAL_VARS.NN_MODEL_NAME in to_skip:
        print "skipping: {}".format(GLOBAL_VARS.NN_MODEL_NAME)
        continue
#     if input_dropout_do:
#         print "skipping: {} due to input dropout".format(GLOBAL_VARS.NN_MODEL_NAME)
#         continue
    
    info('***************************************************************************************')
    info(GLOBAL_VARS.NN_MODEL_NAME)
    
    model = create_keras_nn_model(DOC2VEC_SIZE, NN_OUTPUT_NEURONS, 
                                  first_hidden_layer_size, first_hidden_layer_activation, 
                                  second_hidden_layer_size, second_hidden_layer_activation, 
                                  input_dropout_do, hidden_dropout_do)
    model.summary()
    
    early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=EARLY_STOPPER_MIN_DELTA, \
                                                  patience=EARLY_STOPPER_PATIENCE, verbose=1, mode='auto')
    metrics_callback = MetricsCallback()

    # Model Fitting
    %time history = model.fit(x=X, y=y, validation_data=(Xv,yv), batch_size=4096, \
                              nb_epoch=NN_MAX_EPOCHS, verbose=1, callbacks=[early_stopper, metrics_callback])
    
#     info('Evaluating on Training Data')
#     yp = model.predict(X, batch_size=NN_BATCH_SIZE)
#     yp_binary = get_binary_0_5(yp)
#     #print yp
#     info('Generating Training Metrics')
#     training_metrics = get_metrics(y, yp, yp_binary)
#     print "** Training Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
#         training_metrics['coverage_error'], training_metrics['average_num_of_labels'], 
#         training_metrics['top_1'], training_metrics['top_3'], training_metrics['top_5'], 
#         training_metrics['f1_micro'],training_metrics['f1_macro'],  training_metrics['total_positive'])

    info('Evaluating on Validation Data using last weights')
    yvp = model.predict(Xv)
    yvp_binary = get_binary_0_5(yvp)
    #print yvp
    info('Generating Validation Metrics')
    validation_metrics = get_metrics(yv, yvp, yvp_binary)
    print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
        validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
        validation_metrics['f1_micro'], validation_metrics['f1_macro'])
    last_validation_metrics = validation_metrics

    # using the recorded weights of the best recorded validation loss
    last_model_weights = model.get_weights()
    info('Evaluating on Validation Data using saved best weights')
    model.set_weights(metrics_callback.best_weights)
    yvp = model.predict(Xv)
    yvp_binary = get_binary_0_5(yvp)
    #print yvp
    info('Generating Validation Metrics')
    validation_metrics = get_metrics(yv, yvp, yvp_binary)
    print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
        validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
        validation_metrics['f1_micro'], validation_metrics['f1_macro'])
    best_validation_metrics = validation_metrics

    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME] = dict()
#     param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['training_metrics'] = training_metrics
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_validation_metrics'] = last_validation_metrics
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_validation_metrics'] = best_validation_metrics
#     param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['history'] = history
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['epochs'] = len(history.history['val_loss'])
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_weights'] = metrics_callback.best_weights
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_weights'] = last_model_weights
    
    duration = time.time() - start_time
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['duration'] =  duration
    
    # setting the model weights to the best weights
    model.set_weights(metrics_callback.best_weights)
    
#     ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
#                                              GLOBAL_VARS.NN_MODEL_NAME))
    
#     pickle.dump(model, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
#                                             GLOBAL_VARS.NN_MODEL_NAME, CLASSIFIER), 'w'))
    
#     pickle.dump(best_validation_metrics, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
#                                                           GLOBAL_VARS.NN_MODEL_NAME, VALIDATION_METRICS_FILENAME), 'w'))

    del history, last_model_weights
    
pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

2017-02-02 05:59:48,688 : INFO : ***************************************************************************************
2017-02-02 05:59:48,690 : INFO : nn_1st-size_500_1st-act_relu_2nd-size_2000_2nd-act_sigmoid_in-drop_True_hid-drop_False


skipping: nn_1st-size_200_1st-act_sigmoid_2nd-size_200_2nd-act_sigmoid_in-drop_False_hid-drop_False
skipping: nn_1st-size_200_1st-act_relu_2nd-size_1000_2nd-act_tanh_in-drop_True_hid-drop_False
skipping: nn_1st-size_200_1st-act_softmax_2nd-size_None_2nd-act_sigmoid_in-drop_False_hid-drop_True
skipping: nn_1st-size_500_1st-act_sigmoid_2nd-size_200_2nd-act_softmax_in-drop_False_hid-drop_True
skipping: nn_1st-size_500_1st-act_sigmoid_2nd-size_2000_2nd-act_tanh_in-drop_False_hid-drop_False
skipping: nn_1st-size_200_1st-act_sigmoid_2nd-size_500_2nd-act_relu_in-drop_True_hid-drop_True
skipping: nn_1st-size_500_1st-act_sigmoid_2nd-size_500_2nd-act_tanh_in-drop_False_hid-drop_True
skipping: nn_1st-size_200_1st-act_softmax_2nd-size_200_2nd-act_softmax_in-drop_False_hid-drop_True
skipping: nn_1st-size_500_1st-act_softmax_2nd-size_500_2nd-act_tanh_in-drop_True_hid-drop_True
skipping: nn_1st-size_500_1st-act_relu_2nd-size_50_2nd-act_tanh_in-drop_False_hid-drop_True
skipping: nn_1st-size_200_1st-ac

2017-02-02 06:00:28,257 : INFO : Found lower val loss for epoch 1 => 0.00583


    
1286325/1286325 [==============================] - 34s - loss: 0.0099 - val_loss: 0.0058
Epoch 2/100
1286144/1286325 [============================>.] - ETA: 0s - loss: 0.0059

2017-02-02 06:01:00,260 : INFO : Found lower val loss for epoch 2 => 0.00522


...


Epoch 00035: early stopping
CPU times: user 8min 44s, sys: 10min, total: 18min 45s
Wall time: 18min 16s


2017-02-02 09:48:49,597 : INFO : Generating Validation Metrics
2017-02-02 09:57:06,135 : INFO : Evaluating on Validation Data using saved best weights


****** Validation Metrics: Cov Err: 10.776 | Top 3: 0.631 | Top 5: 0.723 | F1 Micro: 0.331 | F1 Macro: 0.018


2017-02-02 09:58:04,488 : INFO : Generating Validation Metrics


****** Validation Metrics: Cov Err: 10.809 | Top 3: 0.629 | Top 5: 0.722 | F1 Micro: 0.330 | F1 Macro: 0.017


In [38]:
%%time
pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

CPU times: user 11.6 s, sys: 588 ms, total: 12.2 s
Wall time: 12.6 s


In [39]:
print len(param_results_dict)

22


In [42]:
%%time
for key in param_results_dict:
    if param_results_dict[key]['history'] is not None:
        val = param_results_dict[key]
        #param_results_dict[key]['last_weights'] = None
        #param_results_dict[key]['epochs'] = len(val['history'].history['val_loss'])
        param_results_dict[key]['history'] = None
#         param_results_dict[key]['best_weights'] = param_results_dict[key]['metrics_callback'].best_weights
#         param_results_dict[key]['metrics_callback'] = None
        #pickle.dump(param_results_dict[key], open('/mnt/data2/shalaby/nn_parameter_search/doc2vec_size_200_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_14/{}_metrics_callback.pkl'.format(key), 'w'))

pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

CPU times: user 1.06 s, sys: 64 ms, total: 1.12 s
Wall time: 1.11 s


In [38]:
%%time
pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

CPU times: user 3min 25s, sys: 19.3 s, total: 3min 44s
Wall time: 5min 26s


In [47]:
for key in param_results_dict.keys():
    print('========== NN: {}'.format(key))
    val = param_results_dict[key]
    val_metrics = val['last_validation_metrics']
    val_metrics2 =  val['best_validation_metrics']
        
    print('Epochs => {}'.format(val['epochs']))
#     print('Epochs => {}'.format(len(val['history'].history['val_loss'])))
#     print('Best Val Loss => {}'.format(val["metrics_callback"].best_val_loss))
    print('Last Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics['coverage_error'], 
                                                                                        val_metrics['f1_micro'], val_metrics['f1_macro'],
                                                                                        val_metrics['top_3']))
    print('Best Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics2['coverage_error'], 
                                                                                        val_metrics2['f1_micro'], val_metrics2['f1_macro'],
                                                                                        val_metrics2['top_3']))

========== NN: nn_1st-size_200_1st-act_relu_2nd-size_500_2nd-act_softmax_in-drop_False_hid-drop_True
Epochs => 29
Last Val: Coverage Error => 75.0916 | F1 Micro => 0.0000 | F1 Macro => 0.0000 | Top 3 => 0.1867
Best Val: Coverage Error => 75.1124 | F1 Micro => 0.0000 | F1 Macro => 0.0000 | Top 3 => 0.1862
========== NN: nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_softmax_in-drop_False_hid-drop_True
Epochs => 29
Last Val: Coverage Error => 75.0962 | F1 Micro => 0.0000 | F1 Macro => 0.0000 | Top 3 => 0.1862
Best Val: Coverage Error => 75.0842 | F1 Micro => 0.0000 | F1 Macro => 0.0000 | Top 3 => 0.1867
========== NN: nn_1st-size_500_1st-act_tanh_2nd-size_50_2nd-act_sigmoid_in-drop_False_hid-drop_True
Epochs => 86
Last Val: Coverage Error => 8.7398 | F1 Micro => 0.4502 | F1 Macro => 0.0710 | Top 3 => 0.6884
Best Val: Coverage Error => 8.6882 | F1 Micro => 0.4501 | F1 Macro => 0.0732 | Top 3 => 0.6894
========== NN: nn_1st-size_500_1st-act_tanh_2nd-size_200_2nd-act_relu_in-drop_False_h

## Fix old param_results_dict

In [82]:
classifications_type = 'subclasses'

In [83]:
NN_BATCH_SIZE = 4096

In [84]:
epoch = 8
GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)

In [85]:
param_results_dict = pickle.load(open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE)))))

Check if this is an old param_results_dict (it would have history and metrics_callback in that case)

In [86]:
for key in param_results_dict:
    print param_results_dict[key].keys()

['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best_validation_metrics', 'duration', 'best_weights', 'last_weights']
['epochs', 'best

In [71]:
%%time
for key in param_results_dict:
    print param_results_dict[key].keys()
    if param_results_dict[key].get('history') is not None:
        val = param_results_dict[key]
        param_results_dict[key]['epochs'] = len(val['history'].history['val_loss'])
        param_results_dict[key]['best_weights'] = val['metrics_callback'].best_weights
        del param_results_dict[key]['metrics_callback']
        del param_results_dict[key]['history']

pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['last_validation_metrics', 'last_weights', 'best_validation_metrics', 'duration', 'metrics_callback', 'history']
['epochs', 'best_validation_metrics', 'duration', 'best_weights']
['last_validation_metr

## Run network for specific configuration

In [122]:
import sklearn
from sklearn.metrics import coverage_error

In [137]:
def get_metrics_detailed(y_true, y_score, y_binary_score):
    """
    create the metrics object containing all relevant metrics
    """
    metrics = {}
    %time metrics['total_positive'] = np.sum(np.sum(y_binary_score))
    #TODO remove those two when running on the whole set to avoid excessive storage costs
    #metrics['y_true'] = y_true
    #metrics['y_score'] = y_score
    #metrics['y_binary_score'] = y_binary_score
    %time metrics['coverage_error'] = coverage_error(y_true, y_score)
    %time metrics['average_num_of_labels'] = round(float(np.sum(np.sum(y_true, axis=1)))/y_true.shape[0], 2)
    # metrics['average_precision_micro'] = sklearn.metrics.average_precision_score(y_true, y_binary_score, average='micro')
    # metrics['average_precision_macro'] = sklearn.metrics.average_precision_score(y_true, y_binary_score, average='macro')
    #%time metrics['precision_micro'] = sklearn.metrics.precision_score(y_true, y_binary_score, average='micro')
    #%time metrics['precision_macro'] = sklearn.metrics.precision_score(y_true, y_binary_score, average='macro')
    #%time metrics['recall_micro'] = sklearn.metrics.recall_score(y_true, y_binary_score, average='micro')
    #%time metrics['recall_macro'] = sklearn.metrics.recall_score(y_true, y_binary_score, average='macro')
    %time metrics['f1_micro'] = sklearn.metrics.f1_score(y_true, y_binary_score, average='micro')
    %time metrics['f1_macro'] = sklearn.metrics.f1_score(y_true, y_binary_score, average='macro')

    
    if y_true.shape[1] < 100:
        precision_scores = np.zeros(y_true.shape[1])
        for i in range(0, y_true.shape[1]):
            precision_scores[i] = sklearn.metrics.precision_score(y_true[:,i], y_binary_score[:,i])
        metrics['precision_scores_array'] = precision_scores.tolist()
        info('Finished Precision')

        recall_scores = np.zeros(y_true.shape[1])
        for i in range(0, y_true.shape[1]):
            recall_scores[i] = sklearn.metrics.recall_score(y_true[:,i], y_binary_score[:,i])
        metrics['recall_scores_array'] = recall_scores.tolist()
        info('Finished Recall')

        f1_scores = np.zeros(y_true.shape[1])
        for i in range(0, y_true.shape[1]):
            f1_scores[i] = sklearn.metrics.f1_score(y_true[:,i], y_binary_score[:,i])
        metrics['f1_scores_array'] = f1_scores.tolist()
        info('Finished F1')

    %time metrics['top_1'] = get_top_N_percentage(y_score, y_true, max_N=1)
    %time metrics['top_3'] = get_top_N_percentage(y_score, y_true, max_N=3)
    %time metrics['top_5'] = get_top_N_percentage(y_score, y_true, max_N=5)

    return metrics


In [142]:
#EARLY_STOPPER_MIN_DELTA = early_stopper_deltas[classifications_type]
EARLY_STOPPER_MIN_DELTA = 1e-4
EARLY_STOPPER_PATIENCE = early_stopper_patience[classifications_type]
NN_OUTPUT_NEURONS = len(classifications)
NN_BATCH_SIZE = 4096

In [143]:
start_time = time.time()
first_hidden_layer_size = 500
first_hidden_layer_activation = 'tanh'
second_hidden_layer_size = 2000
second_hidden_layer_activation = 'sigmoid'
input_dropout_do = False
hidden_dropout_do = True

#     print "===================================================================================\n" + \
#           "========== 1st Layer Size: {}, 1st Layer Activation: {}, \n 2nd Layer Size: {}, 2nd Layer Activation: {}, \n" + \
#           "Input Dropout: {}, Hidden Dropout: {} \n" + \
#           "==========================".format(first_hidden_layer_size, first_hidden_layer_activation, 
#                                                 second_hidden_layer_size, second_hidden_layer_activation, 
#                                                 input_dropout_do, hidden_dropout_do)

GLOBAL_VARS.NN_MODEL_NAME = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
    first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
    second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
)
if GLOBAL_VARS.NN_MODEL_NAME in param_results_dict.keys():
    print "Should be skipping: {}".format(GLOBAL_VARS.NN_MODEL_NAME)

info('***************************************************************************************')
info(GLOBAL_VARS.NN_MODEL_NAME)

model = create_keras_nn_model(DOC2VEC_SIZE, NN_OUTPUT_NEURONS, 
                              first_hidden_layer_size, first_hidden_layer_activation, 
                              second_hidden_layer_size, second_hidden_layer_activation, 
                              input_dropout_do, hidden_dropout_do)
model.summary()

early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=EARLY_STOPPER_MIN_DELTA, \
                                              patience=EARLY_STOPPER_PATIENCE, verbose=1, mode='auto')
metrics_callback = MetricsCallback()

# Model Fitting
%time history = model.fit(x=X, y=y, validation_data=(Xv,yv), batch_size=NN_BATCH_SIZE, \
                          nb_epoch=NN_MAX_EPOCHS, verbose=1, callbacks=[early_stopper, metrics_callback])

#     info('Evaluating on Training Data')
#     yp = model.predict(X, batch_size=NN_BATCH_SIZE)
#     yp_binary = get_binary_0_5(yp)
#     #print yp
#     info('Generating Training Metrics')
#     training_metrics = get_metrics(y, yp, yp_binary)
#     print "** Training Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
#         training_metrics['coverage_error'], training_metrics['average_num_of_labels'], 
#         training_metrics['top_1'], training_metrics['top_3'], training_metrics['top_5'], 
#         training_metrics['f1_micro'],training_metrics['f1_macro'],  training_metrics['total_positive'])

# info('Evaluating on Validation Data using last weights')
# yvp = model.predict(Xv)
# yvp_binary = get_binary_0_5(yvp)
# #print yvp
# info('Generating Validation Metrics')
# validation_metrics = get_metrics(yv, yvp, yvp_binary)
# print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
#     validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
#     validation_metrics['f1_micro'], validation_metrics['f1_macro'])
# last_validation_metrics = validation_metrics

# using the recorded weights of the best recorded validation loss
last_model_weights = model.get_weights()
info('Evaluating on Validation Data using saved best weights')
model.set_weights(metrics_callback.best_weights)
yvp = model.predict(Xv)
yvp_binary = get_binary_0_5(yvp)
#print yvp
info('Generating Validation Metrics')
validation_metrics = get_metrics_detailed(yv, yvp, yvp_binary)
print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
    validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
    validation_metrics['f1_micro'], validation_metrics['f1_macro'])
best_validation_metrics = validation_metrics

# param_results_dict[GLOBAL_VARS.NN_MODEL_NAME] = dict()
# #     param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['training_metrics'] = training_metrics
# param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_validation_metrics'] = last_validation_metrics
# param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_validation_metrics'] = best_validation_metrics
# param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['history'] = history
# param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['metrics_callback'] = metrics_callback
# param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_weights'] = last_model_weights

# duration = time.time() - start_time
# param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['duration'] =  duration

# pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
#                                        NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

2017-02-15 16:42:45,553 : INFO : ***************************************************************************************
2017-02-15 16:42:45,563 : INFO : nn_1st-size_500_1st-act_tanh_2nd-size_2000_2nd-act_sigmoid_in-drop_False_hid-drop_True


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
doc_input (InputLayer)           (None, 200)           0                                            
____________________________________________________________________________________________________
hidden_layer_tanh (Dense)        (None, 500)           100500      doc_input[0][0]                  
____________________________________________________________________________________________________
dropout_10 (Dropout)             (None, 500)           0           hidden_layer_tanh[0][0]          
____________________________________________________________________________________________________
hidden_layer2_sigmoid (Dense)    (None, 2000)          1002000     dropout_10[0][0]                 
___________________________________________________________________________________________

2017-02-15 16:43:32,610 : INFO : Found lower val loss for epoch 1 => 0.00528


    
1286325/1286325 [==============================] - 32s - loss: 0.0087 - val_loss: 0.0053
Epoch 2/100
 106496/1286325 [=>............................] - ETA: 28s - loss: 0.0044

KeyboardInterrupt: 

## Testing

In [63]:
classifications = valid_subclasses
classifications_type = 'subclasses'

In [64]:
NN_BATCH_SIZE = 4096

In [65]:
param_results_dict = pickle.load(open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE)))))

In [66]:
# Test Metrics
info('Getting Test Vectors Embeddings')
Xt, yt = get_docs_with_inference(doc2vec_model, doc_classification_map, classifications, 
                                               test_docs_list, TEST_MATRIX, 
                                               test_preprocessed_files_prefix, 
                                               test_preprocessed_docids_files_prefix)

2017-02-27 20:42:31,653 : INFO : Getting Test Vectors Embeddings
2017-02-27 20:42:31,670 : INFO : ===== Loading inference vectors
2017-02-27 20:43:04,074 : INFO : Loaded inference vectors matrix


In [67]:
NN_OUTPUT_NEURONS = len(classifications)

In [68]:
param_results_dict.keys()

['nn_1st-size_200_1st-act_tanh_2nd-size_2000_2nd-act_relu_in-drop_False_hid-drop_True',
 'nn_1st-size_1000_1st-act_tanh_2nd-size_2000_2nd-act_sigmoid_in-drop_False_hid-drop_True',
 'nn_1st-size_200_1st-act_tanh_2nd-size_1000_2nd-act_relu_in-drop_False_hid-drop_True',
 'nn_1st-size_500_1st-act_tanh_2nd-size_2000_2nd-act_relu_in-drop_False_hid-drop_True',
 'nn_1st-size_500_1st-act_tanh_2nd-size_2000_2nd-act_sigmoid_in-drop_False_hid-drop_True']

In [69]:
first_hidden_layer_size = 500
first_hidden_layer_activation = 'tanh'
second_hidden_layer_size = 2000
second_hidden_layer_activation = 'relu'
input_dropout_do = False
hidden_dropout_do = True

#     print "===================================================================================\n" + \
#           "========== 1st Layer Size: {}, 1st Layer Activation: {}, \n 2nd Layer Size: {}, 2nd Layer Activation: {}, \n" + \
#           "Input Dropout: {}, Hidden Dropout: {} \n" + \
#           "==========================".format(first_hidden_layer_size, first_hidden_layer_activation, 
#                                                 second_hidden_layer_size, second_hidden_layer_activation, 
#                                                 input_dropout_do, hidden_dropout_do)

GLOBAL_VARS.NN_MODEL_NAME = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
    first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
    second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
)
if GLOBAL_VARS.NN_MODEL_NAME not in param_results_dict.keys():
    print "Can't find model: {}".format(GLOBAL_VARS.NN_MODEL_NAME)
    raise Exception()

info('***************************************************************************************')
info(GLOBAL_VARS.NN_MODEL_NAME)

model = create_keras_nn_model(DOC2VEC_SIZE, NN_OUTPUT_NEURONS, 
                              first_hidden_layer_size, first_hidden_layer_activation, 
                              second_hidden_layer_size, second_hidden_layer_activation, 
                              input_dropout_do, hidden_dropout_do)
model.summary()

# get model best weights
# weights = param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['metrics_callback'].best_weights
weights = param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_weights']
model.set_weights(weights)

info('Evaluating on Test Data using best weights')
ytp = model.predict(Xt)
ytp_binary = get_binary_0_5(ytp)
#print yvp
info('Generating Test Metrics')
test_metrics = get_metrics(yt, ytp, ytp_binary)
print "** Test Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
    test_metrics['coverage_error'], test_metrics['average_num_of_labels'], 
    test_metrics['top_1'], test_metrics['top_3'], test_metrics['top_5'], 
    test_metrics['f1_micro'], test_metrics['f1_macro'], test_metrics['total_positive'])

    
ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
                                             GLOBAL_VARS.NN_MODEL_NAME))

pickle.dump(test_metrics, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
                                                      GLOBAL_VARS.NN_MODEL_NAME, TEST_METRICS_FILENAME), 'w'))


2017-02-27 20:43:35,934 : INFO : ***************************************************************************************
2017-02-27 20:43:35,935 : INFO : nn_1st-size_500_1st-act_tanh_2nd-size_2000_2nd-act_relu_in-drop_False_hid-drop_True
2017-02-27 20:43:36,260 : INFO : Evaluating on Test Data using best weights


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
doc_input (InputLayer)           (None, 200)           0                                            
____________________________________________________________________________________________________
hidden_layer_tanh (Dense)        (None, 500)           100500      doc_input[0][0]                  
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 500)           0           hidden_layer_tanh[0][0]          
____________________________________________________________________________________________________
hidden_layer2_relu (Dense)       (None, 2000)          1002000     dropout_3[0][0]                  
___________________________________________________________________________________________

2017-02-27 20:44:53,016 : INFO : Generating Test Metrics


** Test Metrics: Cov Err: 6.128, Avg Labels: 1.340, 
		 Top 1: 0.618, Top 3: 0.760, Top 5: 0.840, 
		 F1 Micro: 0.554, F1 Macro: 0.186, Total Pos: 331,043


In [62]:
conf_matrix = get_formatted_multilabel_confusion_matrix(yt, ytp_binary, sections)
conf_matrix

,A,B,C,D,E,F,G,H,None
A,46751.0,1541.0,706.0,58.0,187.0,340.0,1275.0,510.0,7508.0
B,1540.0,33737.0,998.0,200.0,387.0,2054.0,2521.0,2261.0,14521.0
C,397.0,975.0,31233.0,70.0,24.0,116.0,719.0,1085.0,6694.0
D,69.0,168.0,56.0,1385.0,4.0,26.0,28.0,33.0,744.0
E,188.0,781.0,47.0,4.0,5118.0,243.0,143.0,151.0,2752.0
F,323.0,1229.0,98.0,37.0,163.0,18152.0,620.0,824.0,6196.0
G,1407.0,1422.0,373.0,29.0,118.0,814.0,118088.0,8379.0,13086.0
H,328.0,604.0,111.0,14.0,59.0,463.0,8070.0,98478.0,11792.0
None,3483.0,3303.0,4579.0,402.0,528.0,1871.0,9905.0,8728.0,0.0


In [65]:
conf_matrix.sum(axis=1)

A        58876.0
B        58219.0
C        41313.0
D         2513.0
E         9427.0
F        27642.0
G       143716.0
H       119919.0
None     32799.0
dtype: float64

In [94]:
conf_matrix.div(conf_matrix.sum(axis=1), axis=0)

,A,B,C,D,E,F,G,H,None
A,0.794059,0.026174,0.011991,0.000985,0.003176,0.005775,0.021656,0.008662,0.127522
B,0.026452,0.579484,0.017142,0.003435,0.006647,0.035281,0.043302,0.038836,0.249420
C,0.009610,0.023600,0.756009,0.001694,0.000581,0.002808,0.017404,0.026263,0.162031
D,0.027457,0.066852,0.022284,0.551134,0.001592,0.010346,0.011142,0.013132,0.296060
E,0.019943,0.082847,0.004986,0.000424,0.542909,0.025777,0.015169,0.016018,0.291927
F,0.011685,0.044461,0.003545,0.001339,0.005897,0.656682,0.022430,0.029810,0.224152
G,0.009790,0.009895,0.002595,0.000202,0.000821,0.005664,0.821676,0.058302,0.091055
H,0.002735,0.005037,0.000926,0.000117,0.000492,0.003861,0.067295,0.821204,0.098333
None,0.106192,0.100704,0.139608,0.012256,0.016098,0.057044,0.301991,0.266106,0.000000


In [90]:
conf_matrix.div(conf_matrix.sum(axis=1), axis=0).round(3)

,A,B,C,D,E,F,G,H,None
A,0.794,0.026,0.012,0.001,0.003,0.006,0.022,0.009,0.128
B,0.026,0.579,0.017,0.003,0.007,0.035,0.043,0.039,0.249
C,0.010,0.024,0.756,0.002,0.001,0.003,0.017,0.026,0.162
D,0.027,0.067,0.022,0.551,0.002,0.010,0.011,0.013,0.296
E,0.020,0.083,0.005,0.000,0.543,0.026,0.015,0.016,0.292
F,0.012,0.044,0.004,0.001,0.006,0.657,0.022,0.030,0.224
G,0.010,0.010,0.003,0.000,0.001,0.006,0.822,0.058,0.091
H,0.003,0.005,0.001,0.000,0.000,0.004,0.067,0.821,0.098
None,0.106,0.101,0.140,0.012,0.016,0.057,0.302,0.266,0.000


In [103]:
format_perc = lambda x: "{:.2f}%".format(x)
(conf_matrix.div(conf_matrix.sum(axis=1), axis=0).round(4) * 100).applymap(format_perc)

,A,B,C,D,E,F,G,H,None
A,79.41%,2.62%,1.20%,0.10%,0.32%,0.58%,2.17%,0.87%,12.75%
B,2.65%,57.95%,1.71%,0.34%,0.66%,3.53%,4.33%,3.88%,24.94%
C,0.96%,2.36%,75.60%,0.17%,0.06%,0.28%,1.74%,2.63%,16.20%
D,2.75%,6.69%,2.23%,55.11%,0.16%,1.03%,1.11%,1.31%,29.61%
E,1.99%,8.28%,0.50%,0.04%,54.29%,2.58%,1.52%,1.60%,29.19%
F,1.17%,4.45%,0.35%,0.13%,0.59%,65.67%,2.24%,2.98%,22.42%
G,0.98%,0.99%,0.26%,0.02%,0.08%,0.57%,82.17%,5.83%,9.11%
H,0.27%,0.50%,0.09%,0.01%,0.05%,0.39%,6.73%,82.12%,9.83%
None,10.62%,10.07%,13.96%,1.23%,1.61%,5.70%,30.20%,26.61%,0.00%


In [66]:
test_metrics.keys()

['recall_macro',
 'f1_scores_array',
 'average_precision_macro',
 'average_num_of_labels',
 'recall_scores_array',
 'average_precision_micro',
 'top_1',
 'precision_micro',
 'top_3',
 'f1_micro',
 'precision_scores_array',
 'f1_macro',
 'recall_micro',
 'coverage_error',
 'top_5',
 'precision_macro',
 'total_positive']

In [72]:
[round(x,2) for x in test_metrics['precision_scores_array']]

[0.86, 0.77, 0.82, 0.63, 0.78, 0.75, 0.84, 0.82]

In [79]:
print 'Precision Scores: {}'.format(' & '.join([str(round(x,2)) for x in test_metrics['precision_scores_array']]))
print 'Recall Scores: {}'.format(' & '.join([str(round(x,2)) for x in test_metrics['recall_scores_array']]))
print 'F1 Scores: {}'.format(' & '.join([str(round(x,2)) for x in test_metrics['f1_scores_array']]))

Precision Scores: 0.86 & 0.77 & 0.82 & 0.63 & 0.78 & 0.75 & 0.84 & 0.82
Recall Scores: 0.79 & 0.58 & 0.76 & 0.55 & 0.54 & 0.66 & 0.82 & 0.82
F1 Scores: 0.82 & 0.66 & 0.79 & 0.59 & 0.64 & 0.7 & 0.83 & 0.82


In [60]:
ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
                                             GLOBAL_VARS.NN_MODEL_NAME))
pickle.dump(test_metrics, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, 
                                                      GLOBAL_VARS.NN_MODEL_NAME, TEST_METRICS_FILENAME), 'w'))


In [51]:
test_metrics = get_metrics(yt, ytp, ytp_binary)
print "** Test Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
    test_metrics['coverage_error'], test_metrics['average_num_of_labels'], 
    test_metrics['top_1'], test_metrics['top_3'], test_metrics['top_5'], 
    test_metrics['f1_micro'], test_metrics['f1_macro'], test_metrics['total_positive'])


/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true s

** Test Metrics: Cov Err: 2.955, Avg Labels: 1.240, 
		 Top 1: 0.699, Top 3: 0.850, Top 5: 0.911, 
		 F1 Micro: 0.648, F1 Macro: 0.169, Total Pos: 350,752


In [141]:
for key in param_results_dict.keys():
    print('========== NN: {}'.format(key))
    val = param_results_dict[key]
    val_metrics = val['last_validation_metrics']
    val_metrics2 =  val['best_validation_metrics']
    
    print('Epochs => {}'.format(len(val['history'].history['val_loss'])))
    print('Best Val Loss => {}'.format(val["metrics_callback"].best_val_loss))
    print('Last Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics['coverage_error'], 
                                                                                        val_metrics['f1_micro'], val_metrics['f1_macro'],
                                                                                        val_metrics['top_3']))
    print('Best Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics2['coverage_error'], 
                                                                                        val_metrics2['f1_micro'], val_metrics2['f1_macro'],
                                                                                        val_metrics2['top_3']))

========== NN: nn_1st-size_500_1st-act_softmax_2nd-size_None_2nd-act_softmax_in-drop_True_hid-drop_False
Epochs => 100
Best Val Loss => 0.166829405505
Last Val: Coverage Error => 1.4960 | F1 Micro => 0.7434 | F1 Macro => 0.6293 | Top 3 => 0.9637
Best Val: Coverage Error => 1.4960 | F1 Micro => 0.7434 | F1 Macro => 0.6293 | Top 3 => 0.9637
========== NN: nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_relu_in-drop_False_hid-drop_True
Epochs => 47
Best Val Loss => 0.147426413828
Last Val: Coverage Error => 1.4576 | F1 Micro => 0.7738 | F1 Macro => 0.7021 | Top 3 => 0.9697
Best Val: Coverage Error => 1.4576 | F1 Micro => 0.7738 | F1 Macro => 0.7021 | Top 3 => 0.9697
========== NN: nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_tanh_in-drop_True_hid-drop_True
Epochs => 28
Best Val Loss => 0.192695072037
Last Val: Coverage Error => 1.6299 | F1 Micro => 0.6631 | F1 Macro => 0.4731 | Top 3 => 0.9455
Best Val: Coverage Error => 1.6299 | F1 Micro => 0.6631 | F1 Macro => 0.4731 | Top 3 => 0

In [109]:
for key in param_results_dict.keys():
    print('========== NN: {}'.format(key))
    val = param_results_dict[key]
    val_metrics = val['validation_metrics']
    val_metrics2 =  val['metrics_callback'].metrics_dict[sorted(val['metrics_callback'].metrics_dict.keys())[-1]]
    
    print('Epochs => {}'.format(len(val['history'].history['val_loss'])))
    print('Best Val Loss => {}'.format(val["metrics_callback"].best_val_loss))
    print('Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics['coverage_error'], 
                                                                                        val_metrics['f1_micro'], val_metrics['f1_macro'],
                                                                                        val_metrics['top_3']))
    print('Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics2['coverage_error'], 
                                                                                        val_metrics2['f1_micro'], val_metrics2['f1_macro'],
                                                                                        val_metrics2['top_3']))

========== NN: nn_1st-size_500_1st-act_sigmoid_2nd-size_None_2nd-act_relu_in-drop_True_hid-drop_True
Epochs => 52
Best Val Loss => 0.209273911595
Coverage Error => 1.6301 | F1 Micro => 0.6278 | F1 Macro => 0.4057 | Top 3 => 0.9440
Coverage Error => 1.6337 | F1 Micro => 0.6286 | F1 Macro => 0.4065 | Top 3 => 0.9448
========== NN: nn_1st-size_200_1st-act_sigmoid_2nd-size_None_2nd-act_relu_in-drop_True_hid-drop_True
Epochs => 35
Best Val Loss => 0.218694725994
Coverage Error => 1.6793 | F1 Micro => 0.5939 | F1 Macro => 0.3668 | Top 3 => 0.9360
Coverage Error => 1.6744 | F1 Micro => 0.5924 | F1 Macro => 0.3680 | Top 3 => 0.9370
========== NN: nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_softmax_in-drop_False_hid-drop_True
Epochs => 79
Best Val Loss => 0.161037461469
Coverage Error => 1.5147 | F1 Micro => 0.7480 | F1 Macro => 0.6480 | Top 3 => 0.9605
Coverage Error => 1.5151 | F1 Micro => 0.7483 | F1 Macro => 0.6455 | Top 3 => 0.9606
========== NN: nn_1st-size_200_1st-act_tanh_2nd-size

In [163]:
pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))